In [179]:
import numpy as np
import pandas as pd
import datetime

### Cleaning

In [180]:
elo = pd.read_csv('./data/nfl_elo_csv.csv')
games = pd.read_csv('./data/all_games.csv')

#### simple cleaning tasks
  * remove old games from elo csv (only have games from 2009 onward in all_games.csv) 
  * drop na values from games (e.g., na for score field)

In [181]:
# only keep data from 2009 onward in ELO (because thats all game data we have)
start = datetime.datetime(2009, 1, 1) 
elo['date'] = pd.to_datetime(elo['date'])
elo = elo.rename(columns={"team1": "home_team", "team2": "away_team"})


elo = elo[elo['date'] > start]
elo.head()

,date,season,neutral,playoff,home_team,away_team,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,score1,score2
13593,2009-01-03,2008,0,w,lac,ind,1593.456,1691.593,0.452456,0.547544,1615.091,1669.958,23,17
13594,2009-01-03,2008,0,w,ari,atl,1493.926,1558.510,0.500599,0.499401,1513.358,1539.078,30,24
13595,2009-01-04,2008,0,w,mia,bal,1545.051,1644.229,0.450972,0.549028,1518.900,1670.380,9,27
13596,2009-01-04,2008,0,w,min,phi,1583.539,1616.177,0.546438,0.453562,1555.089,1644.627,14,26
13597,2009-01-10,2008,0,d,car,ari,1586.798,1513.358,0.689316,0.310684,1542.007,1558.150,13,33


In [182]:
# drop any nas in games
games = games[(games['home_score'].notna()) & (games['away_score'].notna())]
games.head()

,type,game_id,home_team,away_team,week,season,state_of_game,game_url,home_score,away_score,Unnamed: 0
0,post,2017010700,HOU,OAK,18,2016,POST,http://www.nfl.com/liveupdate/game-center/2017...,27.0,14.0,NaN
1,post,2017010701,SEA,DET,18,2016,POST,http://www.nfl.com/liveupdate/game-center/2017...,26.0,6.0,NaN
2,post,2017010800,PIT,MIA,18,2016,POST,http://www.nfl.com/liveupdate/game-center/2017...,30.0,12.0,NaN
3,post,2017010801,GB,NYG,18,2016,POST,http://www.nfl.com/liveupdate/game-center/2017...,38.0,13.0,NaN
4,post,2017011400,ATL,SEA,18,2016,POST,http://www.nfl.com/liveupdate/game-center/2017...,36.0,20.0,NaN


### Ensure team abbreviations have 1:1 mapping across datasets

In [183]:
# need to ensure team mappings are 1:1 across datasets
elo_teams = elo['home_team'].unique()
elo_teams = [w.upper() for w in elo_teams]
games_teams = games['home_team'].unique()

# find differences 
elo_diff = set(elo_teams).difference(set(games_teams)) # whats in elo and not in games
game_diff = set(games_teams).difference(set(elo_teams)) # whats in games and not elo
print(f'whats in elo and not in games: {elo_diff}\n')
print(f'whats in games and not elo: {game_diff}\n')



whats in elo and not in games: {'WSH', 'LAR'}

whats in games and not elo: {'WAS', 'LA', 'RIC', 'JAC', 'STL', 'APR', 'NPR', 'SD', 'CRT', 'SAN'}



#### Mappings that need to be fixed
 * SD --> chargers (games)
 * JAC --> jaguars (games)
 * LA --> rams (games)
 * WAS --> commanders (games)
 * STL --> rams (games) 

 * map rams to --> LAR
 * map commanders to --> WSH
 * map chargers to --> LAC

 * drop APR, CRT, NPR, RIC, SAN (games)

 * ELO dataset is clean
 * LAR --> rams (elo)
 * WSH --> commanders (elo)

In [184]:
nonsense_teams = ['APR', 'CRT', 'NPR', 'RIC', 'SAN', 'CRT']
games = games[(~games['home_team'].isin(nonsense_teams)) & (~games['away_team'].isin(nonsense_teams))]
# convert elo abbreviations to upper
elo['home_team'] = elo['home_team'].apply(str.upper)
elo['away_team'] = elo['away_team'].apply(str.upper)


In [185]:
# correct mapping abbreviations
def correct_mapping(name):
    if name == 'LA'  or name == 'STL': # rams
        return 'LAR'
    elif name == 'WAS':
        return 'WSH'
    elif name == 'JAC':
        return 'JAX'
    elif name == 'SD':
        return 'LAC'
    else:
        return name

games['home_team'] = games['home_team'].apply(correct_mapping)
games['away_team'] = games['away_team'].apply(correct_mapping)

# sanity check
elo_teams = elo['home_team'].unique()
games_teams = games['home_team'].unique()
elo_diff = set(elo_teams).difference(set(games_teams)) # whats in elo and not in games
game_diff = set(games_teams).difference(set(elo_teams)) # whats in games and not elo
print(f'whats in elo and not in games: {elo_diff}\n')
print(f'whats in games and not elo: {game_diff}\n')

whats in elo and not in games: set()

whats in games and not elo: set()



### Join ELO and games datasets using dates and team abbreviations

In [186]:
# create date column in games dataset
# parse date from url
def date_from_url(url):
    """
    each game queried from sports-center api
    date found at ...center/<date>/... portion of url
    date in form of YYYYMMDDXX where XX is not needed
    date-form has len 8
    """
    i = url.index('center') + len('center') + 1 # +1 because of /
    date = url[i:i+8]
    return date

games['date'] = games['game_url'].apply(date_from_url)
games['date'] = pd.to_datetime(games['date'])

In [188]:
complete_df = pd.merge(games, elo, left_on=['date','home_team'], right_on = ['date','home_team'])

In [189]:
complete_df = complete_df.drop('Unnamed: 0', axis=1)

In [191]:
# Y refers to elo dataset, X refers to game dataset (y, x) appended if identical column names
# validate no na values except for playoffs column
complete_df.isna().sum()

type                0
game_id             0
home_team           0
away_team_x         0
week                0
season_x            0
state_of_game       0
game_url            0
home_score          0
away_score          0
date                0
season_y            0
neutral             0
playoff          2293
away_team_y         0
elo1_pre            0
elo2_pre            0
elo_prob1           0
elo_prob2           0
elo1_post           0
elo2_post           0
score1              0
score2              0
dtype: int64

In [ ]:
complete_df.to_csv('elo_games_full.csv')